# Politics in Urbana

In [2]:
SCHOOL="urbana"
SUBJECT="opinions"
START_YEAR=2010
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [3]:
%cd ..

/home/alexandersuen/diversity-colleges


In [4]:
%cd src

/home/alexandersuen/diversity-colleges/src


In [5]:
import sys
sys.path.append('src')
import ouraws
import nltk
import ourgraphs
import textutil

In [6]:
%cd ..

/home/alexandersuen/diversity-colleges


In [7]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [8]:
df.shape

(4226, 6)

In [9]:
df.head(2)

,title,url,body,year,month,day
0,Opinion | Celebrities need to stop lying about...,https://dailyillini.com/opinions-stories/2023/...,"If you’ve been on any social media page, chanc...",2023,2,28
1,"Opinion | Microchips are the present, question...",https://dailyillini.com/opinions-stories/2023/...,Dystopian novels might have predicted the futu...,2023,2,27


In [10]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")
#BIPARTISAN_API_KEY = "gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=="
print(BIPARTISAN_API_KEY)
BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g==


In [11]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-17.110746


In [12]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010])

In [13]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2010 ... 151
2010	151	-368.93894700000004	-2.443304284768212
Year: 2011 ... 333
2011	325	-597.5630929999996	-1.8386556707692296
Year: 2012 ... 157
2012	153	-135.667891	-0.8867182418300653
Year: 2013 ... 195
2013	186	297.94689199999993	1.6018650107526877
Year: 2014 ... 670
2014	657	1223.2566830000005	1.861882318112634
Year: 2015 ... 441
2015	434	-196.2487550000005	-0.4521860714285726
Year: 2016 ... 394
2016	390	-584.712152	-1.499261928205128
Year: 2017 ... 346
2017	345	-1302.87181	-3.7764400289855073
Year: 2018 ... 339
2018	334	-627.5885660000004	-1.8790076826347315
Year: 2019 ... 351
2019	347	-530.3288770000004	-1.5283252939481278
Year: 2020 ... 300
2020	300	-1011.394928	-3.3713164266666666
Year: 2021 ... 282
2021	279	-737.1507399999999	-2.6421173476702506
Year: 2022 ... 237
2022	236	-1101.926956	-4.6691820169491525


In [14]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)

In [ ]:
# ASYNCHRONOUS METHOD : should be a lot faster
import asyncio

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year